In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  mean_squared_error, make_scorer
from monk_helpers import read_ds
import skorch
import torch
import torch.nn as nn
from monk_helpers import CV


In [ ]:
# Datasets Path
TR_PATH = "./monks/datasets/monks-1.train"
TS_PATH = "./monks/datasets/monks-1.test"

In [ ]:


class Net(nn.Module):

  def __init__(self, input_size=17, neurons=5, output_size=1,hidden_activation_fun = nn.Tanh,output_activation_fun =nn.Sigmoid):
    super().__init__()
    self.neurons = neurons
    self.hidden_activation_fun = hidden_activation_fun()
    self.output_activation_fun = output_activation_fun()   
    self.l1 = nn.Linear(input_size, neurons)
    self.l2 = nn.Linear(neurons, output_size)
    
  def forward(self, x):
    out = self.hidden_activation_fun(self.l1(x))
    out = self.output_activation_fun(self.l2(out))
    return out
  
  def reset_weights(net):
      for param in net.parameters():
          torch.nn.init.uniform_(param, a=-0.7, b=0.7)



In [ ]:
X_train, y_train = read_ds(TR_PATH)
X_test, y_test = read_ds(TS_PATH)
X_train

In [ ]:

model_nn = NeuralNetClassifier(
    module=Net,

    callbacks=[skorch.callbacks.EarlyStopping(monitor='valid_loss', patience=50, threshold=1e-4)]
)
params_grid = {
    'batch_size': [10, 20, 40, 60, 80, 100],
    'max_epochs': [4, 32, 64],
    "criterion":[nn.MSELoss,nn.BCELoss, nn.BCEWithLogitsLoss],
    'optimizer': [optim.SGD, optim.RMSprop, optim.Adagrad, optim.Adadelta,
                  optim.Adam, optim.Adamax, optim.NAdam],
    'module__hidden_activation_fun': [nn.Identity, nn.ReLU, nn.ELU, nn.ReLU6,
                           nn.GELU, nn.Softplus, nn.Softsign, nn.Tanh],
    'module__output_activation_fun': [nn.Sigmoid,nn.Softmax],
    'optimizer__lr': [0.01, 0.2, 0.25, 0.3],
    'optimizer__momentum': [0.5, 0.6, 0.7],
    "model__neurons": [2, 3, 4],
}

'''params_grid = {
    'batch_size': [8],
    'max_epochs': [600],
    "criterion":  [nn.MSELoss],
    'optimizer': [optim.SGD],
    'module__hidden_activation_fun': [nn.Tanh],
    'module__output_activation_fun': [nn.Sigmoid],
    'optimizer__lr': [0.5],
    'optimizer__momentum': [0.7],
    "module__neurons": [4],
    'module__input_size':[len(X_train.columns)],
    'module__output_size':[1]
}'''

custom_scores_monk = {
    "accuracy": "accuracy",
    "mse": make_scorer(mean_squared_error,greater_is_better=False)
}

grid = GridSearchCV(model_nn,
                    param_grid=params_grid,
                    cv=CV,
                    refit=True,
                    return_train_score=True,
                    n_jobs=4,
        )

In [ ]:
X = torch.tensor(X_train.values, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)

In [ ]:
grid.fit(X, y)
print("Best parameters: " + str(grid.best_params_) + " score: " + str(grid.best_score_))

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report


y_pred = grid.best_estimator_.predict(X_test.values.astype("float32"))
#y_pred = y_pred.round().int().view(-1)
print("accuracy on test set {:.3f}".format(accuracy_score(y_test,y_pred)))
print(classification_report(y_test, 
                            y_pred, 
                            target_names=['0', '1']))

#print the confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()
plt.show()
print("------------------------------------------------------------------------------------------------------")